# Preprocess c file

TCI percentiles per city-country

`Last update`: 2021-04-15

In [86]:
import subprocess
import pandas as pd

def _create_cs_file(path_c):
    
    df = pd.read_csv(path_c) \
        .drop_duplicates('region_slug') \
        .dropna()

    for k in df.columns:
        if k != 'region_slug':
            df[k] = pd.to_numeric(df[k])
        if k == 'region_slug':
            df[k] = df[k].astype(str) 

    df_c = df \
        .assign(c_low_p01 = lambda x: (x['q25']-x['q01'])/(x['q75']-x['q25']),
                c_upp_p99 = lambda x: (x['q99']-x['q75'])/(x['q75']-x['q25']),
                c_low_p05 = lambda x: (x['q25']-x['q05'])/(x['q75']-x['q25']),
                c_upp_p95 = lambda x: (x['q95']-x['q75'])/(x['q75']-x['q25'])
               ) \
        [['region_slug', 'c_low_p01', 'c_upp_p99', 'c_low_p05', 'c_upp_p95']]
    
    return df_c

def _save_cs_file(): 
    
    df_c = _create_cs_file(path_c = '~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_iqr.csv')
    print(df_c.shape)
    print(df_c.describe())
    df_c.to_csv('~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_c_iqr.csv', index=False)
    

## Copy percentile

In [81]:
!cp ~/shared/spd-sdv-omitnik-waze/waze_tci/region_quantiles/quantiles/regions_quantiles.csv ~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/raw/regions_quantiles.csv

!sed "s/\[//g;s/\]//g;s/['\"]//g" ~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/raw/regions_quantiles.csv > ~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_iqr_00.csv

In [82]:
%%bash 

echo -e "region_slug,mean,var_p, var_s,q10,q20,q30,q40,q50,q60,q70,q80,q90,q01,q99,q25,q75,q05,q95" | cat - ~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_iqr_00.csv > ~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_iqr.csv

In [83]:
!rm ~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_iqr_00.csv

In [84]:
!head -2 ~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_iqr.csv

region_slug,mean,var_p, var_s,q10,q20,q30,q40,q50,q60,q70,q80,q90,q01,q99,q25,q75,q05,q95
aguascalientes,8039136.868743886,8862843413565.455,8887326406420.607,3829261.1993683483,5605996.229991711,6600013.058713919,6932325.881132038,7473441.530126513,8626943.901942156,9718372.086622398,11136396.36923769,11136396.36923769,2151065.466800198,14393234.680959623,6215087.576892198,10248917.983242217,3349729.7486610203,12964231.367048915


## Save c's file

In [87]:
_save_cs_file()

(44, 5)
       c_low_p01  c_upp_p99  c_low_p05  c_upp_p95
count  44.000000  44.000000  44.000000  44.000000
mean    0.820025   1.366873   0.598703   0.741052
std     0.424155   0.643011   0.341299   0.317299
min     0.213163   0.553016   0.165875   0.308937
25%     0.508470   0.868821   0.351556   0.508035
50%     0.643371   1.214658   0.482633   0.634525
75%     1.098601   1.649564   0.747647   0.895277
max     1.766687   3.101491   1.580424   1.567010


In [88]:
df = pd.read_csv('~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_c_iqr.csv')

In [89]:
df

,region_slug,c_low_p01,c_upp_p99,c_low_p05,c_upp_p95
0,aguascalientes,1.007485,1.027390,0.710332,0.673135
1,arequipa,1.370780,1.318878,0.967781,0.620649
2,asuncion,0.615222,1.589508,0.503198,0.604833
3,barcelona,0.564193,1.984999,0.447251,1.037351
4,barquisimeto,0.590701,1.705025,0.494875,0.761201
5,barranquilla,1.766687,1.225445,1.580424,0.717043
6,belem,0.675131,0.638890,0.402509,0.368560
7,belohorizonte,0.510229,0.793227,0.347080,0.491304
8,belpoman,0.355745,2.239444,0.247018,1.252462
9,brasilia,0.454875,1.045578,0.313769,0.506300
